### Requirements/Libraries


In [1]:
from rdflib import Graph
from SPARQLWrapper import SPARQLWrapper, JSON

### Query local Playground dataset

In [2]:
g = Graph()
g.parse("playground.ttl", format="ttl")

print("Loaded '" + str(len(g)) + "' triples.")

print("Query results (Females):")

qres = g.query(
"""SELECT ?thing ?name where {
  ?thing tto:sex "female" .
  ?thing dbp:name ?name .
}""")

for row in qres:
    #Row is a list of matched RDF terms: URIs, literals or blank nodes
    print("%s is female with name '%s'" % (str(row.thing),str(row.name)))

Loaded '73' triples.
Query results (Females):
http://example.org/tuto/resource#Eve is female with name 'Eve'
http://example.org/tuto/resource#LunaCat is female with name 'Luna'


In [26]:
g = Graph()
g.parse("playground.ttl", format="ttl")

print("Loaded '" + str(len(g)) + "' triples.")

print("Query results (Eve's father):")

qres = g.query(
"""SELECT ?father ?name where {
  ttr:Eve dbo:parent ?father .
  ?father dbp:name ?name
}""")

for row in qres:
    #Row is a list of matched RDF terms: URIs, literals or blank nodes
    print("%s is Eve's father with name '%s'" % (str(row.father),str(row.name)))

Loaded '73' triples.
Query results (Eve's father):
http://example.org/tuto/resource#William is Eve's father with name 'William'


### Query 4.1

In [27]:
g = Graph()
g.parse("playground.ttl", format="ttl")

print("Loaded '" + str(len(g)) + "' triples.")

print("Query results (Eve's grandfather):")

qres = g.query(
"""SELECT ?grandfather ?name where {
  ttr:Eve dbo:parent / dbo:parent ?grandfather .
  ?grandfather dbp:name ?name
}""")

for row in qres:
    #Row is a list of matched RDF terms: URIs, literals or blank nodes
    print("%s is Eve's grandfather with name '%s'" % (str(row.grandfather),str(row.name)))

Loaded '73' triples.
Query results (Eve's grandfather):
http://example.org/tuto/resource#John is Eve's grandfather with name 'John'


### Select person who don't have cat

In [53]:
g = Graph()
g.parse("playground.ttl", format="ttl")

print("Loaded '" + str(len(g)) + "' triples.")

print("Query results (Person with no cat):")

qres = g.query(
"""select ?person ?name where {
    ?person rdf:type dbo:Person .
	filter not exists {
    ?person tto:pet / rdf:type tto:Cat .
    }
}""")

for row in qres:
    #Row is a list of matched RDF terms: URIs, literals or blank nodes
    print("%s is person with no cat with name" % (str(row.person)))

Loaded '73' triples.
Query results (Person with no cat):
http://example.org/tuto/resource#Eve is person with no cat with name
http://example.org/tuto/resource#William is person with no cat with name


### Query 4.2

### Query local Nobel Prize dataset

In [3]:
g = Graph()
g.parse("../files/nobelprize_kg.nt", format="nt")
  
print("Loaded '" + str(len(g)) + "' triples.")

#Query with prefixes 
nobelprize_query = """
PREFIX nobel: <http://data.nobelprize.org/terms/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
SELECT DISTINCT ?name_laur WHERE { 
?laur rdf:type nobel:Laureate . 
?laur rdfs:label ?name_laur . 
?laur foaf:gender "female" . }
"""

qres = g.query(nobelprize_query)

print("Female laureates (printing 10):")
i=0
for row in qres:
    #Row is a list of matched RDF terms: URIs, literals or blank nodes
    i+=1
    print("'%s'" % (str(row.name_laur))) #Same name as in sparql query SELECT variable
    if i==10:
        break

Loaded '85797' triples.
Female laureates (printing 10):
'Marie Curie, née Sklodowska'
'Maria Goeppert Mayer'
'Irène Joliot-Curie'
'Dorothy Crowfoot Hodgkin'
'Gerty Theresa Cori, née Radnitz'
'Rosalyn Yalow'
'Barbara McClintock'
'Rita Levi-Montalcini'
'Gertrude B. Elion'
'Christiane Nüsslein-Volhard'


### Query remote dataset via SPARQL Enpoint

In [4]:
endpoint_url = "http://dbpedia.org/sparql"
sparqlw = SPARQLWrapper(endpoint_url)
sparqlw.setReturnFormat(JSON)


query = """
PREFIX foaf: <http://xmlns.com/foaf/0.1/> 
PREFIX dbo: <http://dbpedia.org/ontology/> 
SELECT DISTINCT ?x WHERE { 
?jd foaf:name \"Johnny Depp\"@en . 
?m dbo:starring ?jd .
?m dbo:starring ?other . 
?other foaf:name ?x . 
FILTER (STR(?x)!=\"Johnny Depp\")
} 
ORDER BY ?x
"""

sparqlw.setQuery(query)

results = sparqlw.query().convert()

#Prints JSON file
#print(results)   

print("Stars co-starring Johnny Depp (printing 10):")
i=0
for result in results["results"]["bindings"]:
    i+=1
    #Prints individual results 
    print(result["x"]["value"])
    if i==10:
        break

Stars co-starring Johnny Depp (printing 10):

Abigail Breslin
Adam Scott
Aidan Quinn
Al Pacino
Alain Chabat
Alan Arkin
Albert Tsai
Alfred Molina
Alison Sudol
